! jt -r

In [259]:
from importlib import reload
reload(aux)
reload(con)
reload(dt)
reload (src.density)

NameError: name 'src' is not defined

In [265]:
import pandas as pd
import src.daily_data as dt
import src.auxiliary_functions as aux
import src.condition as con
from datetime import time, datetime
import src.info as info
import src.in_out as io
import numpy as np
from src.density import Density

1. **data** klasöründeki tüm dosya yolları alınıyor

In [2]:
path_list = io.get_path('C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\')

2. *Her bir path'de bulunan veriler **dataframe** olarak alınıyor. Her bir df, **split_df** metodu ile saatlik olarak **23** ayrı df'e bölünüyor. Bölünmüş her bir df'in **mid_price**'ı hesaplanıp yeni bir sütun olarak **df**'e ekleniyor.* **get_info** metodu her bir path için bu verileri kullanarak bir **info** örneği oluşturup döndürür. Her bir **info** örneği, aşağıdaki özelliklere sahiptir:<br>
    * Okunan dosyanın tarihi
    * Ürünün adı
    * Okunan dosyaya ait yol
    * Tüm verileri içeren dataframe
    * Saatlik olarak bölünmüş dataframe listesi


In [3]:
infolist = [info.get_info(path) for path in path_list]

3. Burada, oluşturulan infolar arasında bir **eşleştirme** yapılmaktadır. Çünkü her bir **spread** hesabının hangi veriler kullanılarak yapılacağı bilinmelidir. Eşleşen infolar, tuple çiftleri olarak tutulmakta.
  Elimizde 2018:09:06 tarihinin 6A,6B,6C ürünlerine ait verileri olsun. Bu durumda 6A-6B, 6A-6C, 6B-6C arasında 3 adet spread hesaplanacaktır. Hesaplama öncesi bu eşleştirme, **daily_data** modülünün **to_match_days** metoduyla yapılmaktadır.

In [4]:
paired_days = dt.to_match_days(infolist)

4.  **daily_datas**: GünlükVeri örneklerinin tutulduğu liste <br />
**dt.to_match_hour** metodu ile her bir eşleşmiş çiftin mid_price'ları saatlik olarak alınıp birleştiriliyor. <br />
**gunluk_veri**: **tarih**, her bir saate ait **change**,**spread**,**amplitude**,**duration**,**urun adları**...vs içeriyor <br />
**paired_hours:** 23 adet df içerir. her bir df mid_price çifti içerir

In [5]:
daily_data = []

In [6]:
for pair in paired_days:
    product = pair[0].product + "_" + pair[1].product   # urun adi
    paired_hours=dt.to_match_hour(pair[0],pair[1])
    gunluk_veri = dt.DailyData(mid_price_list=paired_hours,date=pair[0].date, name= product)
    gunluk_veri.spread_hesapla()
    daily_data.append(gunluk_veri)

5. Her bir **DailyData** örneği ürünlere göre ayrı ayrı gruplandırılıyor. <br>
    (['**6AU8_6BU8**': [gunluk veriler...], '**6AU8_6CU8**':[gunluk veriler...], '**6BU8_6CU8**':[gunluk veriler...]])

In [7]:
products={}

In [8]:
for gunluk in daily_data:
    if gunluk.name in products:
        products[gunluk.name].append(gunluk)
    else:
        products[gunluk.name] = [gunluk]

**combined_days :**  key-value olarak ürün adları ve verileri içerir <br />
**combined_days_detail :** key-value olarak ürün adları ve verilere ait istatistiksel bilgileri içerir <br />
**combined_days_con :** key-value olarak ürün adları ve median koşuluna göre yeniden düzenlenmiş verileri içerir <br />
* 6BU8_6CU8'e ait verilere erişmek için:<br />
 ```python
combined_days_detail['6BU8_6CU8']
```
* 6BU8_6CU8'e ait 18:00:00-19:00:00 saatleri arası verilere erişmek için: <br />
 ```python
combined_days['6BU8_6CU8'].loc[time(18)]
```

In [9]:
combined_days = {}
combined_days_detail = {}
combined_days_con = {}
combined_days_con_detail = {}

In [10]:
stop=False
i=0
while products:
    urun = products.popitem()
    df = dt.get_all_data(urun[1]).dropna()
    con_df = con.scan(df)
    combined_days[urun[0]] = df
    combined_days_con[urun[0]] = con_df
#    combined_days_detail[urun[0]] = aux.get_detail(df)
#     combined_days_con_detail[urun[0]] = aux.get_detail(con_df)

Hesaplanmış ürün adları ve saat bilgisi alınıyor

In [12]:
all_products = list(combined_days.keys())

In [13]:
hour_series = pd.date_range('2018-01-01-18', periods=23, freq='H').time

all_density:
```json
{"6BU8_6CU8": {"18:00:00": "Density Object", "19:00:00": "Density Object"...},
 "6AU8_6CU8":{"18:00:00": "Density Object", "19:00:00": "Density Object"...},
 ....}
```

In [261]:
all_density= {}

In [262]:
for product in all_products:
    densities = {}
    for hour in hour_series:
        df = combined_days[product].loc[hour]
        densities[hour]= den.Density(df)
    all_density[product] = densities

--------------------------------

-4.999999999999449 şeklinde amplitude 5'e yuvarlanıyor

# Example

In [183]:
data_of_the_eighteenth_clock = all_density['6AU8_6BU8'][time(18)]

In [184]:
df = data_of_the_eighteenth_clock.density_data

In [231]:
df_piv = pd.pivot_table(df, values='density', index=['duration'],columns='amplitude',margins=True)

In [239]:
df_piv

amplitude,-13.125000000003638,-13.125000000002718,-11.87500000001819,-11.875000000016733,-11.250000000025466,-11.25000000002374,-10.0,-9.999999999998899,-9.375000000007276,-9.375000000005906,...,9.999999999998899,10.0,10.000000000010001,10.000000000010914,13.124999999991616,13.124999999992724,20.624999999990788,20.624999999992724,31250.0,All
duration,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.004016,0.004016,NaN,NaN,...,0.008032,0.008032,NaN,NaN,NaN,NaN,NaN,NaN,0.004016,0.012894
2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.005355
3.0,NaN,NaN,NaN,NaN,0.004016,0.004016,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004685
4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.004016,0.004016,NaN,NaN,NaN,NaN,NaN,0.008032
5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.005355
6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004016
7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.012048
8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.005622
9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004016


In [252]:
df_piv_d = df_piv.drop('All',axis=1)

In [253]:
density = (df_piv_d.iloc[:-1]/ df_piv_d.iloc[-1]).stack().to_frame('marginal_distribution')


In [254]:
density

marginal_distribution
duration amplitude                        
1.0      -10.000                  1.000000
         -10.000                  1.000000
         -6.250                   1.000000
         -6.250                   1.000000
         -5.000                   1.000000
         -5.000                   1.000000
         -3.125                   5.151515
         -3.125                   5.151515
          1.875                   1.000000
          1.875                   1.000000
          3.125                   4.551724
          3.125                   4.551724
          5.000                   1.666667
          5.000                   1.666667
          6.250                   1.000000
          6.250                   1.000000
          10.000                  1.333333
          10.000                  1.333333
          31250.000               1.000000
2.0      -8.125                   1.000000
         -8.125                   1.000000
         -8.125                   1.000000
         -8.125                   1.000000
         -6.250                   1.000000
         -6.250                   1.000000
         -3.125                   1.030303
         -3.125                   1.030303
          3.125                   1.241379
          3.125                   1.241379
          3.125                   1.000000
...                                    ...
63.0      8.125                   1.000000
          8.125                   1.000000
69.0     -3.125                   0.515152
         -3.125                   0.515152
83.0      3.125                   0.413793
          3.125                   0.413793
88.0     -6.250                   1.000000
         -6.250                   1.000000
108.0    -3.125                   0.515152
         -3.125                   0.515152
110.0     6.250                   1.000000
          6.250                   1.000000
113.0     9.375                   1.000000
          9.375                   1.000000
121.0     6.250                   1.000000
          6.250                   1.000000
135.0    -3.125                   0.515152
         -3.125                   0.515152
136.0     8.125                   1.000000
          8.125                   1.000000
138.0    -10.000                  1.000000
         -10.000                  1.000000
154.0    -3.125                   0.515152
         -3.125                   0.515152
155.0    -6.250                   1.000000
         -6.250                   1.000000
210.0     20.625                  1.000000
          20.625                  1.000000
416.0     3.125                   1.000000
          3.125                   1.000000

[179 rows x 1 columns]

In [119]:
a= df_piv.groupby('duration')

In [81]:
df_piv.agg(['sum'])

,-13.125000000003638,-13.125000000002718,-11.87500000001819,-11.875000000016733,-11.250000000025466,-11.25000000002374,-10.0,-9.999999999998899,-9.375000000007276,-9.375000000005906,...,9.999999999998899,10.0,10.000000000010001,10.000000000010914,13.124999999991616,13.124999999992724,20.624999999990788,20.624999999992724,31250.0,All
sum,0.008032,0.008032,0.008032,0.008032,0.008032,0.008032,0.024096,0.024096,0.008032,0.008032,...,0.018072,0.018072,0.008032,0.008032,0.008032,0.008032,0.008032,0.008032,0.008032,0.216205


In [108]:
df_piv.agg(['sum']).transpose()

,sum
-13.125,0.004016
-13.125,0.004016
-11.875,0.004016
-11.875,0.004016
-11.250,0.004016
-11.250,0.004016
-10.000,0.020080
-10.000,0.020080
-9.375,0.004016
-9.375,0.004016


data_of_the_eighteenth_clock.data

data_of_the_eighteenth_clock.density_data

In [ ]:
data_of_the_eighteenth_clock.draw()

In [236]:
path = 'C:\\Users\\ugur.eren\\Python Codes\\cefis2\\out\\'

In [250]:
writer = pd.ExcelWriter(path+ 'density'+'.xlsx', engine='xlsxwriter')

In [251]:
density.to_excel(writer, sheet_name='Sheet1')
writer.save()
